In [2]:
import random
import matplotlib.pyplot as plt
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [61]:

def create_data_model(waypoints):
    data = {}
    data['distance_matrix'] = compute_distance_matrix(waypoints)
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

def compute_distance_matrix(waypoints):
    n = len(waypoints)
    distance_matrix = []
    for i in range(n):
        row = []
        for j in range(n):
            dx = waypoints[i][0] - waypoints[j][0]
            dy = waypoints[i][1] - waypoints[j][1]
            distance = int((dx**2 + dy**2)**0.5)
            row.append(distance)
        distance_matrix.append(row)
    return distance_matrix

def draw_circle(ax, center, radius):
    circle = plt.Circle(center, radius, color='b', fill=False)
    ax.add_artist(circle)
    
def print_solution(manager, routing, solution):
    print('TSP Distance: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = ' \n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    return plan_output



In [4]:
def angle_between_points(p1, p2):
    return np.arctan2(p2[1] - p1[1], p2[0] - p1[0])
import numpy as np

def angle_between_points(p1, p2):
    return np.arctan2(p2[1] - p1[1], p2[0] - p1[0])

def point_on_circle(center, angle, radius):
    x = center[0] + radius * np.cos(angle)
    y = center[1] + radius * np.sin(angle)
    return (x, y)

In [62]:
def plot_solution(waypoints, plan_output, radius):
    ordered_points = [waypoints[int(node)] for node in plan_output.split() if node.isdigit()]
    ordered_points.append(ordered_points[0])

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(*zip(*ordered_points))

    for i, point in enumerate(ordered_points[:-1]):
        plt.annotate(i, (point[0] + 0.5, point[1] + 0.5))
        if i > 0 and i < len(ordered_points) - 2:  # Exclude start and end points from circles
            draw_circle(ax, point, radius)

    new_path = [ordered_points[0]]
    for i in range(1, len(ordered_points) - 2):
        # calculate angle and chords for the current waypoint
        angle = angle_between_points(ordered_points[i-1], ordered_points[i])
        nextangle = angle_between_points(ordered_points[i], ordered_points[i+1])
        chord_start = point_on_circle(ordered_points[i], angle - np.pi , radius)
        chord_end = point_on_circle(ordered_points[i], nextangle , radius)

        # calculate the chord for the previous waypoint
        prev_angle = angle_between_points(ordered_points[i - 2] if i > 1 else ordered_points[0], ordered_points[i-1])
        prev_chord_end = point_on_circle(ordered_points[i-1], prev_angle + np.pi / 2, radius)

        # add the point on the current waypoint's circle that is closest to the previous waypoint
        if np.linalg.norm(np.array(chord_start) - np.array(prev_chord_end)) < np.linalg.norm(np.array(chord_end) - np.array(prev_chord_end)):
            new_path.extend([chord_start, chord_end])
            ax.plot(*zip(*[chord_start, chord_end]), 'r-')  # visualize the chord
        else:
            new_path.extend([chord_end, chord_start])
            ax.plot(*zip(*[chord_end, chord_start]), 'r-')  # visualize the chord

    # add the ending point
    new_path.append(ordered_points[-1])
    total_distance = sum(np.linalg.norm(np.array(new_path[i]) - np.array(new_path[i+1])) for i in range(len(new_path) - 1))
    print(f"Total distance of new path: {total_distance:.2f}")
    ax.plot(*zip(*new_path), linestyle='-', marker='o')
    

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('TSP Solution')
    plt.grid()
    plt.axis('equal')
    plt.show()
    return total_distance

In [6]:
# import matplotlib.pyplot as plt
# import numpy as np

# def plot_solution(waypoints, plan_output, radius):
#     ordered_points = [waypoints[int(node)] for node in plan_output.split() if node.isdigit()]
#     # ordered_points.append(ordered_points[0])

#     fig, ax = plt.subplots(figsize=(10, 6))
    

#     ax.scatter(*zip(*ordered_points),color='yellow',s=80)
#     ax.scatter(ordered_points[0][0], ordered_points[0][1], color='red',s=120)
#     ax.annotate('Home base', (ordered_points[0][0], ordered_points[0][1]),
#             textcoords="offset points", xytext=(0,10), ha='center', va='bottom', color='white')

#     for i, point in enumerate(ordered_points[:-1]):
#         ax.annotate(i, (point[0] + 0.5, point[1] + 0.5), color='blue', fontsize=10, ha='center', va='center')
#         if i > 0 and i < len(ordered_points) - 2:  # Exclude start and end points from circles
#             draw_circle(ax, point, radius)

#     new_path = [ordered_points[0]]
#     for i in range(1, len(ordered_points) - 2):
#         # calculate angle and chords for the current waypoint
#         angle = angle_between_points(ordered_points[i-1], ordered_points[i])
#         nextangle = angle_between_points(ordered_points[i], ordered_points[i+1])
#         chord_start = point_on_circle(ordered_points[i], angle - np.pi , radius)
#         chord_end = point_on_circle(ordered_points[i], nextangle , radius)

#         # calculate the chord for the previous waypoint
#         prev_angle = angle_between_points(ordered_points[i - 2] if i > 1 else ordered_points[0], ordered_points[i-1])
#         prev_chord_end = point_on_circle(ordered_points[i-1], prev_angle + np.pi / 2, radius)

#         # add the point on the current waypoint's circle that is closest to the previous waypoint
#         if np.linalg.norm(np.array(chord_start) - np.array(prev_chord_end)) < np.linalg.norm(np.array(chord_end) - np.array(prev_chord_end)):
#             new_path.extend([chord_start, chord_end])
#             # ax.plot(*zip(*[chord_start, chord_end]), 'r-')  # visualize the chord
#         else:
#             new_path.extend([chord_end, chord_start])
#             # ax.plot(*zip(*[chord_end, chord_start]), 'r-')  # visualize the chord

#     # add the ending point
#     new_path.append(ordered_points[-1])
#     total_distance = sum(np.linalg.norm(np.array(new_path[i]) - np.array(new_path[i+1])) for i in range(len(new_path) - 1))
#     print(f"Total distance of new path: {total_distance:.2f}")

#     for i in range(len(new_path) - 1):
#         x, y = new_path[i]
#         dx = new_path[i+1][0] - x
#         dy = new_path[i+1][1] - y
#         ax.arrow(x, y, dx, dy, head_width=0.5, head_length=0.8, lw=1.5)
#     ax.axis('off')
#     ax.set_facecolor('none')
#     fig.set_facecolor('none')
#     plt.xlabel('X')
#     plt.ylabel('Y')
#     plt.title('Interest Point', color='white')
#     plt.axis('equal')
#     plt.show()
#     return total_distance

# # Please ensure the remaining helper functions like draw_circle, angle_between_points, and point_on_circle are defined.


In [63]:
def main(waypoints):
    data = create_data_model(waypoints)
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        plan_output = print_solution(manager, routing, solution)
        # plot_solution(waypoints, plan_output, radius)
        return plan_output
    return None
if __name__ == '__main__':
    waypoints = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(20)]
    waypoints.insert(0, (0, 0))
    # waypoints = [(0,0),(10,10),(17,3),(20,12.5)] # 4
    # # waypoints = [(0,0),(10,10),(15,3),(20,12.5),(30,2),(10,5)] # 6 
    # # waypoints = [(0,0),(10,10),(17,3),(20,12.5),(30,-2.5),(10,-5),(30,14),(40,10)] # 8 
    # # waypoints = [(0,0),(10,10),(17,3),(20,12.5),(30,-2.5),(10,-5),(30,14),(40,10),(40,-10),(20,-10)]# 10 

    # waypoints = [(0, 0),(11, 54),(4, 49),(69, 26),(12, 39)] #5
    # waypoints = [(0, 0),(11, 54),(4, 49),(69, 26),(12, 39),(25, 1)] #6
    # waypoints = [(0, 0),(11, 54),(4, 49),(69, 26),(12, 39),(25, 1),(17, 12),(38, 11),(28, 49)] #9
    # waypoints = [(0, 0),(11, 54),(4, 49),(69, 26),(12, 39),(25, 1),(17, 12),(38, 11),(28, 49),(52, 36),(36, 27)] #11
    # waypoints = [(0, 0),(11, 54),(4, 49),(69, 26),(12, 39),(25, 1),(17, 12),(38, 11),(28, 49),(52, 36),(36, 27),(56, 44),(22, 58)] #13
    # waypoints = [(0, 0),(11, 54),(4, 49),(69, 26),(12, 39),(25, 1),(17, 12),(38, 11),(28, 49),(52, 36),(36, 27)] #10
    
    # waypoints = [(0, 0),(54, 85),(8, 92),(25, 32),(71, 1),(6, 75),(21, 87),(35, 48),(13, 55),(75, 44),(91, 95),(62, 77),(53, 20),(79, 70),(40, 7)] # 15 

    radius = 5
    
    plan_output = main(waypoints)
    # print(f"waypoints =  {waypoints}")
    # print(f"plan_output =  {plan_output}")
    

TSP Distance: 462 miles
 
 0 -> 14 -> 16 -> 6 -> 8 -> 9 -> 12 -> 4 -> 1 -> 15 -> 10 -> 13 -> 3 -> 11 -> 2 -> 20 -> 17 -> 7 -> 19 -> 18 -> 5 -> 0



In [84]:
import time
import csv
import math
waypoints =  [(0, 0), (82, 13), (95, 78), (97, 10), (55, 32), (29, 52), (63, 31), (79, 74), (97, 52), (72, 58), (33, 33), (46, 59), (64, 38), (47, 95), (95, 53), (69, 40), (25, 74), (59, 96), (77, 51), (60, 27), (47, 51)]
speed = 10
def distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)


In [88]:

# Write the header of the CSV file just once
with open('TSP_time.csv', 'w', newline='') as csvfile:
    fieldnames = ['Runtime', 'waypointN','TSPOut','waypoints','ordered_points',"missiontime"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

for i in range(0,12):
    start_time = time.time()
    subset_waypoints = waypoints[0:11+i]
    plan_output = main(subset_waypoints)
    end_time = time.time()
    Runtime = end_time - start_time
    # print(subset_waypoints)
    print(f"TSP took {Runtime:.4f} seconds to execute.with number of waypints {10+i}")
    waypointN = 10+i

    total_time = 0
    ordered_points = [waypoints[int(node)] for node in plan_output.split() if node.isdigit()]
    # print(ordered_points)
    for i in range(len(ordered_points) - 1):
    # print(ordered_points[i], ordered_points[i+1])
        total_time += distance(ordered_points[i], ordered_points[i+1]) / speed
        total_time += 1/6  # add waiting time at each waypoint
    total_time -= 1/6
    print(len(ordered_points)-2,total_time)
    
    # result = find_min_total_time_node_at_level(root,len(waypoints))
    # total_time = result['total_time']
    
    # Open the CSV file in append mode to avoid overwriting
    with open('TSP_time.csv', 'a', newline='') as csvfile:
        fieldnames = ['Runtime', 'waypointN','TSPOut','waypoints','ordered_points','missiontime']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the data
        writer.writerow({'Runtime': Runtime, 'waypointN': waypointN,'TSPOut' : plan_output,'waypoints':subset_waypoints,'ordered_points': ordered_points,'missiontime':total_time})


TSP Distance: 320 miles
 
 0 -> 10 -> 4 -> 6 -> 1 -> 3 -> 8 -> 2 -> 7 -> 9 -> 5 -> 0

TSP took 0.0018 seconds to execute.with number of waypints 10
10 33.98802213146616
TSP Distance: 321 miles
 
 0 -> 10 -> 4 -> 6 -> 1 -> 3 -> 8 -> 2 -> 7 -> 9 -> 11 -> 5 -> 0

TSP took 0.0015 seconds to execute.with number of waypints 11
11 34.253430126250905
TSP Distance: 330 miles
 
 0 -> 10 -> 4 -> 12 -> 6 -> 1 -> 3 -> 8 -> 2 -> 7 -> 9 -> 11 -> 5 -> 0

TSP took 0.0016 seconds to execute.with number of waypints 12
12 35.40264318191346
TSP Distance: 384 miles
 
 0 -> 10 -> 4 -> 12 -> 6 -> 1 -> 3 -> 8 -> 2 -> 7 -> 9 -> 13 -> 11 -> 5 -> 0

TSP took 0.0018 seconds to execute.with number of waypints 13
13 41.03419881575562
TSP Distance: 385 miles
 
 0 -> 10 -> 4 -> 12 -> 6 -> 1 -> 3 -> 8 -> 14 -> 2 -> 7 -> 9 -> 13 -> 11 -> 5 -> 0

TSP took 0.0020 seconds to execute.with number of waypints 14
14 41.31679131809121
TSP Distance: 391 miles
 
 0 -> 10 -> 4 -> 6 -> 12 -> 15 -> 1 -> 3 -> 8 -> 14 -> 2 -> 7 -> 9 -

In [79]:
waypoints =  [(0, 0), (82, 13), (95, 78), (97, 10), (55, 32), (29, 52), (63, 31), (79, 74), (97, 52), (72, 58), (33, 33), (46, 59), (64, 38), (47, 95), (95, 53), (69, 40), (25, 74), (59, 96), (77, 51), (60, 27), (47, 51)]